In [ ]:
## This file implements neural networks and logistic regression on p0017presabsSTCC_quant.
## For fully-connected neural networks, the accuracy is 57.14%.
## To improve accuracy, we construct lasso regression to select important features, and thus form a new dataset.
## With the new dataset, the accuracy is 61.04%, and keeps 61.04% after improvement.
## For logistic regression, the accuracy is 61.04%.
## We further construct random forest models, which are relatively less likely to bring overfitting 
## compared to decision tree.
## For random forest, the accuracy is 63.64%.
## For random forest with cross-validation, the mean accuracy is 77.48%.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p0017presabsSTCC_quant.csv')
df.shape

(255, 7006)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0.871375
1      0.364375
2      0.803000
3      0.952625
4      1.012875
5      0.936750
6      0.745375
7      0.348375
8      0.571750
9      0.637125
10     0.670500
11     0.731250
12     0.865125
13     0.569375
14     0.327000
15     0.175750
16     0.964000
17     1.104167
18     0.223667
19     0.340167
20     0.438500
21     0.271833
22     0.248833
23     0.739333
24     0.232833
25     0.601500
26     0.251000
27     0.202833
28     0.599833
29     0.260667
         ...   
225    0.459500
226    0.825333
227    0.763167
228    0.334500
229    0.998833
230    0.491125
231    0.405750
232    0.875375
233    0.761500
234    0.255000
235    0.455125
236    0.234125
237    0.340125
238    0.952750
239    0.238500
240    0.294500
241    0.172125
242    0.221500
243    0.989875
244    0.763000
245    0.229875
246    0.215125
247    0.805375
248    0.184875
249    0.663250
250    0.803125
251    0.443875
252    0.207500
253    0.933875
254    0.873375
Name: pheno, Length: 255

In [5]:
df.head()

,id,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,...,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno,ST,CC
0,107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.871375,5,5
1,109,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.364375,8,8
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.803000,5,5
3,120335,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.952625,5,5
4,120337,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1.012875,5,5


In [6]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [7]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [8]:
df['pheno']
df['pheno'].value_counts()

1    135
0    120
Name: pheno, dtype: int64

In [9]:
df.shape

(255, 7006)

In [10]:
df_clean = df.drop(columns=['id'])

In [11]:
df_clean.shape

(255, 7005)

In [12]:
df_clean.head()

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,...,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno,ST,CC
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,5,5
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,8
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,5,5
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,5,5


In [13]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 7004) (255,)


In [14]:
# split into train, validation, and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y)

In [15]:
############# Fully-Connected Neural Network ################

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

Using TensorFlow backend.


In [17]:
#### neural network
model1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [18]:
model1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
model1.fit(X_train, y_train,
          batch_size=32, epochs=100,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/100
178/178 [==============================] - 0s 752us/step - loss: 13.9673 - accuracy: 0.5281 - val_loss: 31.8429 - val_accuracy: 0.5455
Epoch 2/100
178/178 [==============================] - 0s 219us/step - loss: 5.9862 - accuracy: 0.5787 - val_loss: 0.6472 - val_accuracy: 0.5325
Epoch 3/100
178/178 [==============================] - 0s 219us/step - loss: 0.6208 - accuracy: 0.7135 - val_loss: 0.6436 - val_accuracy: 0.5195
Epoch 4/100
178/178 [==============================] - 0s 210us/step - loss: 0.6240 - accuracy: 0.7247 - val_loss: 0.6464 - val_accuracy: 0.5195
Epoch 5/100
178/178 [==============================] - 0s 245us/step - loss: 0.5895 - accuracy: 0.6910 - val_loss: 0.6530 - val_accuracy: 0.5065
Epoch 6/100
178/178 [==============================] - 0s 222us/step - loss: 0.5790 - accuracy: 0.7079 - val_loss: 0.6406 - val_accuracy: 0.5325
Epoch 7/100
178/178 [==============================] - 0s 216us/step - loss: 0.5734

Epoch 57/100
178/178 [==============================] - 0s 249us/step - loss: 0.4456 - accuracy: 0.8146 - val_loss: 0.6563 - val_accuracy: 0.5974
Epoch 58/100
178/178 [==============================] - 0s 221us/step - loss: 0.4475 - accuracy: 0.8090 - val_loss: 0.6579 - val_accuracy: 0.5974
Epoch 59/100
178/178 [==============================] - 0s 335us/step - loss: 0.4424 - accuracy: 0.8371 - val_loss: 0.6712 - val_accuracy: 0.5844
Epoch 60/100
178/178 [==============================] - 0s 422us/step - loss: 0.4381 - accuracy: 0.8371 - val_loss: 0.6649 - val_accuracy: 0.6234
Epoch 61/100
178/178 [==============================] - 0s 208us/step - loss: 0.4471 - accuracy: 0.8034 - val_loss: 0.6615 - val_accuracy: 0.5844
Epoch 62/100
178/178 [==============================] - 0s 195us/step - loss: 0.4365 - accuracy: 0.8427 - val_loss: 0.6742 - val_accuracy: 0.5974
Epoch 63/100
178/178 [==============================] - 0s 197us/step - loss: 0.4354 - accuracy: 0.8258 - val_loss: 0.6647 -

In [20]:
acc_test = model1.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test*100))

77/77 [==============================] - 0s 201us/step
test accuracy: 57.14%


In [49]:
#### improve neural network
model2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [50]:
model2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [51]:
model2.fit(X_train, y_train,
          batch_size=64, epochs=1000,
          validation_data=(X_test, y_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 879us/step - loss: 9.9520 - accuracy: 0.4213 - val_loss: 0.7277 - val_accuracy: 0.5065
Epoch 2/1000
178/178 [==============================] - 0s 248us/step - loss: 0.6989 - accuracy: 0.5225 - val_loss: 0.6739 - val_accuracy: 0.5325
Epoch 3/1000
178/178 [==============================] - 0s 214us/step - loss: 0.6687 - accuracy: 0.6236 - val_loss: 0.6508 - val_accuracy: 0.5325
Epoch 4/1000
178/178 [==============================] - 0s 203us/step - loss: 0.6453 - accuracy: 0.6854 - val_loss: 1.2424 - val_accuracy: 0.4935
Epoch 5/1000
178/178 [==============================] - 0s 205us/step - loss: 1.0453 - accuracy: 0.6011 - val_loss: 0.6342 - val_accuracy: 0.5195
Epoch 6/1000
178/178 [==============================] - 0s 196us/step - loss: 0.6286 - accuracy: 0.6742 - val_loss: 0.6288 - val_accuracy: 0.5195
Epoch 7/1000
178/178 [==============================] - 0s 228us/step - loss: 0

178/178 [==============================] - 0s 197us/step - loss: 0.5149 - accuracy: 0.7528 - val_loss: 0.6303 - val_accuracy: 0.5714
Epoch 57/1000
178/178 [==============================] - 0s 243us/step - loss: 0.5134 - accuracy: 0.7809 - val_loss: 0.6325 - val_accuracy: 0.5714
Epoch 58/1000
178/178 [==============================] - 0s 275us/step - loss: 0.5137 - accuracy: 0.7697 - val_loss: 0.6297 - val_accuracy: 0.5584
Epoch 59/1000
178/178 [==============================] - 0s 225us/step - loss: 0.5097 - accuracy: 0.7753 - val_loss: 0.6296 - val_accuracy: 0.5714
Epoch 60/1000
178/178 [==============================] - 0s 209us/step - loss: 0.5093 - accuracy: 0.7921 - val_loss: 0.6298 - val_accuracy: 0.5714
Epoch 61/1000
178/178 [==============================] - 0s 202us/step - loss: 0.5055 - accuracy: 0.7921 - val_loss: 0.6307 - val_accuracy: 0.5714
Epoch 62/1000
178/178 [==============================] - 0s 191us/step - loss: 0.5042 - accuracy: 0.7809 - val_loss: 0.6298 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 169us/step - loss: 0.4342 - accuracy: 0.8315 - val_loss: 0.6488 - val_accuracy: 0.5844
Epoch 113/1000
178/178 [==============================] - 0s 158us/step - loss: 0.4323 - accuracy: 0.8258 - val_loss: 0.6484 - val_accuracy: 0.5844
Epoch 114/1000
178/178 [==============================] - 0s 163us/step - loss: 0.4302 - accuracy: 0.8315 - val_loss: 0.6474 - val_accuracy: 0.5974
Epoch 115/1000
178/178 [==============================] - 0s 169us/step - loss: 0.4289 - accuracy: 0.8315 - val_loss: 0.6476 - val_accuracy: 0.6104
Epoch 116/1000
178/178 [==============================] - 0s 151us/step - loss: 0.4271 - accuracy: 0.8258 - val_loss: 0.6458 - val_accuracy: 0.5974
Epoch 117/1000
178/178 [==============================] - 0s 195us/step - loss: 0.4267 - accuracy: 0.8258 - val_loss: 0.6481 - val_accuracy: 0.5974
Epoch 118/1000
178/178 [==============================] - 0s 193us/step - loss: 0.4242 - accuracy: 0.8315 - val_

178/178 [==============================] - 0s 198us/step - loss: 0.3801 - accuracy: 0.8539 - val_loss: 0.6798 - val_accuracy: 0.6104
Epoch 168/1000
178/178 [==============================] - 0s 228us/step - loss: 0.3792 - accuracy: 0.8596 - val_loss: 0.6761 - val_accuracy: 0.6104
Epoch 169/1000
178/178 [==============================] - 0s 252us/step - loss: 0.3818 - accuracy: 0.8483 - val_loss: 0.6729 - val_accuracy: 0.5974
Epoch 170/1000
178/178 [==============================] - 0s 205us/step - loss: 0.3811 - accuracy: 0.8315 - val_loss: 0.6784 - val_accuracy: 0.6104
Epoch 171/1000
178/178 [==============================] - 0s 189us/step - loss: 0.3793 - accuracy: 0.8427 - val_loss: 0.6821 - val_accuracy: 0.5974
Epoch 172/1000
178/178 [==============================] - 0s 215us/step - loss: 0.3818 - accuracy: 0.8596 - val_loss: 0.6783 - val_accuracy: 0.5974
Epoch 173/1000
178/178 [==============================] - 0s 230us/step - loss: 0.3773 - accuracy: 0.8483 - val_loss: 0.6779 - 

178/178 [==============================] - 0s 223us/step - loss: 0.3393 - accuracy: 0.8708 - val_loss: 0.7168 - val_accuracy: 0.5844
Epoch 223/1000
178/178 [==============================] - ETA: 0s - loss: 0.3929 - accuracy: 0.87 - 0s 231us/step - loss: 0.3438 - accuracy: 0.8708 - val_loss: 0.7528 - val_accuracy: 0.5974
Epoch 224/1000
178/178 [==============================] - 0s 210us/step - loss: 0.3755 - accuracy: 0.8539 - val_loss: 0.7186 - val_accuracy: 0.5844
Epoch 225/1000
178/178 [==============================] - 0s 203us/step - loss: 0.3638 - accuracy: 0.8652 - val_loss: 0.7970 - val_accuracy: 0.5714
Epoch 226/1000
178/178 [==============================] - 0s 232us/step - loss: 0.3874 - accuracy: 0.8258 - val_loss: 0.7062 - val_accuracy: 0.5974
Epoch 227/1000
178/178 [==============================] - 0s 341us/step - loss: 0.3393 - accuracy: 0.8820 - val_loss: 0.7083 - val_accuracy: 0.5974
Epoch 228/1000
178/178 [==============================] - 0s 299us/step - loss: 0.345

...

178/178 [==============================] - 0s 186us/step - loss: 0.2157 - accuracy: 0.8933 - val_loss: 1.0364 - val_accuracy: 0.5584
Epoch 773/1000
178/178 [==============================] - 0s 181us/step - loss: 0.2179 - accuracy: 0.8933 - val_loss: 1.1583 - val_accuracy: 0.5714
Epoch 774/1000
178/178 [==============================] - 0s 170us/step - loss: 0.2221 - accuracy: 0.9045 - val_loss: 1.0012 - val_accuracy: 0.5844
Epoch 775/1000
178/178 [==============================] - 0s 161us/step - loss: 0.2121 - accuracy: 0.8933 - val_loss: 1.0347 - val_accuracy: 0.5584
Epoch 776/1000
178/178 [==============================] - 0s 192us/step - loss: 0.2500 - accuracy: 0.8876 - val_loss: 1.0974 - val_accuracy: 0.5584
Epoch 777/1000
178/178 [==============================] - 0s 177us/step - loss: 0.2201 - accuracy: 0.8933 - val_loss: 1.0170 - val_accuracy: 0.5584
Epoch 778/1000
178/178 [==============================] - 0s 179us/step - loss: 0.2128 - accuracy: 0.8989 - val_loss: 1.0273 - 

178/178 [==============================] - 0s 153us/step - loss: 0.2483 - accuracy: 0.8989 - val_loss: 1.0775 - val_accuracy: 0.5714
Epoch 828/1000
178/178 [==============================] - 0s 193us/step - loss: 0.2451 - accuracy: 0.8876 - val_loss: 1.0372 - val_accuracy: 0.5584
Epoch 829/1000
178/178 [==============================] - 0s 189us/step - loss: 0.2500 - accuracy: 0.8764 - val_loss: 1.1671 - val_accuracy: 0.5844
Epoch 830/1000
178/178 [==============================] - 0s 172us/step - loss: 0.2532 - accuracy: 0.8820 - val_loss: 1.1815 - val_accuracy: 0.5714
Epoch 831/1000
178/178 [==============================] - 0s 210us/step - loss: 0.2417 - accuracy: 0.8820 - val_loss: 1.0788 - val_accuracy: 0.5584
Epoch 832/1000
178/178 [==============================] - 0s 191us/step - loss: 0.2362 - accuracy: 0.8820 - val_loss: 1.2302 - val_accuracy: 0.5844
Epoch 833/1000
178/178 [==============================] - 0s 165us/step - loss: 0.2841 - accuracy: 0.8820 - val_loss: 1.0649 - 

178/178 [==============================] - 0s 213us/step - loss: 0.2217 - accuracy: 0.8876 - val_loss: 1.0618 - val_accuracy: 0.5584
Epoch 883/1000
178/178 [==============================] - 0s 207us/step - loss: 0.2010 - accuracy: 0.8989 - val_loss: 1.0942 - val_accuracy: 0.5584
Epoch 884/1000
178/178 [==============================] - 0s 235us/step - loss: 0.2016 - accuracy: 0.8989 - val_loss: 1.0847 - val_accuracy: 0.5584
Epoch 885/1000
178/178 [==============================] - 0s 185us/step - loss: 0.2251 - accuracy: 0.8933 - val_loss: 1.2290 - val_accuracy: 0.5714
Epoch 886/1000
178/178 [==============================] - 0s 231us/step - loss: 0.2760 - accuracy: 0.8652 - val_loss: 1.0707 - val_accuracy: 0.5714
Epoch 887/1000
178/178 [==============================] - 0s 373us/step - loss: 0.2165 - accuracy: 0.8933 - val_loss: 1.1388 - val_accuracy: 0.5584
Epoch 888/1000
178/178 [==============================] - 0s 183us/step - loss: 0.2015 - accuracy: 0.9045 - val_loss: 1.0730 - 

178/178 [==============================] - 0s 210us/step - loss: 0.2215 - accuracy: 0.9045 - val_loss: 1.0866 - val_accuracy: 0.5584
Epoch 938/1000
178/178 [==============================] - 0s 227us/step - loss: 0.2068 - accuracy: 0.8989 - val_loss: 1.0495 - val_accuracy: 0.5714
Epoch 939/1000
178/178 [==============================] - 0s 184us/step - loss: 0.2063 - accuracy: 0.8989 - val_loss: 1.0506 - val_accuracy: 0.5714
Epoch 940/1000
178/178 [==============================] - 0s 234us/step - loss: 0.2267 - accuracy: 0.8933 - val_loss: 1.0775 - val_accuracy: 0.5714
Epoch 941/1000
178/178 [==============================] - 0s 219us/step - loss: 0.2089 - accuracy: 0.8933 - val_loss: 1.2336 - val_accuracy: 0.5714
Epoch 942/1000
178/178 [==============================] - 0s 255us/step - loss: 0.2218 - accuracy: 0.8933 - val_loss: 1.0880 - val_accuracy: 0.5584
Epoch 943/1000
178/178 [==============================] - 0s 240us/step - loss: 0.2033 - accuracy: 0.8989 - val_loss: 1.0917 - 

Epoch 992/1000
178/178 [==============================] - 0s 202us/step - loss: 0.2071 - accuracy: 0.8933 - val_loss: 1.1880 - val_accuracy: 0.5584
Epoch 993/1000
178/178 [==============================] - 0s 196us/step - loss: 0.2131 - accuracy: 0.8933 - val_loss: 1.0616 - val_accuracy: 0.5584
Epoch 994/1000
178/178 [==============================] - 0s 199us/step - loss: 0.2059 - accuracy: 0.8933 - val_loss: 1.0556 - val_accuracy: 0.5714
Epoch 995/1000
178/178 [==============================] - 0s 250us/step - loss: 0.2251 - accuracy: 0.8876 - val_loss: 1.1399 - val_accuracy: 0.5584
Epoch 996/1000
178/178 [==============================] - 0s 177us/step - loss: 0.2617 - accuracy: 0.8764 - val_loss: 1.0914 - val_accuracy: 0.5714
Epoch 997/1000
178/178 [==============================] - 0s 193us/step - loss: 0.2145 - accuracy: 0.8876 - val_loss: 1.1387 - val_accuracy: 0.5584
Epoch 998/1000
178/178 [==============================] - 0s 203us/step - loss: 0.2052 - accuracy: 0.9045 - val_

In [46]:
acc_test2 = model2.evaluate(X_test, y_test)[1]
print('test accuracy: %.2f%%' % (acc_test2*100))

77/77 [==============================] - 0s 177us/step
test accuracy: 55.84%


In [52]:
############ Feature selection using lasso ##########

In [53]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [54]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [55]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [56]:
X_train_features = np.vstack((names, X_train))
X_train_features = pd.DataFrame(X_train_features)

In [57]:
sel_features = X_train_features.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 7004
selected features: 110


In [58]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 354,  403,  570,  672,  861,  874,  910,  991,  994,  998, 1372,
        1576, 1606, 1947, 2089, 2104, 2106, 2235, 2283, 2292, 2319, 2326,
        2481, 2511, 2520, 2546, 2587, 2630, 2666, 2804, 2830, 2997, 3013,
        3109, 3354, 3584, 3586, 3613, 3623, 3686, 3752, 3817, 3825, 3874,
        4031, 4047, 4096, 4227, 4278, 4344, 4345, 4350, 4449, 4463, 4544,
        4545, 4568, 4623, 4645, 4665, 4736, 4738, 4762, 4842, 5047, 5068,
        5100, 5133, 5143, 5189, 5382, 5492, 5543, 5639, 5658, 5745, 5758,
        5890, 5955, 5982, 6005, 6024, 6067, 6192, 6215, 6393, 6421, 6424,
        6533, 6547, 6581, 6596, 6616, 6745, 6755, 6914, 6926, 6933, 6935,
        6941, 6960, 6965, 6972, 6976, 6980, 6983, 6984, 6986, 7002, 7003]])

In [59]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTGGAACAAAAC',
       'TTTGAATGCTAACGTAACTATTTCCATCTCTTCGTTTAACTAAGTTGAATTTGCCCATACCTGATTCAATTGTAGTGTCTCCGCCTGTAGCATAGTCACC',
       'TTTATCTATATCA',
       'TTTACAATTGCTTATAAGAAATCTGAAACATTTAGAAATTTTGTTAATGGTGCAATTGAAAGTGTTAAACAAACATTTAGTAATTTTATTCAATTTATTC',
       'TTGGAGTAATT',
       'TTGCTTATAAGAAATCTGAAACATTTAGAAATTTTGTTAATGGTGCAATTGAAAGTGTTAAACAAACATTTAGTAATTTTATTCAATTTATTCAACCTTT',
       'TTGCCAGGATA',
       'TTGAATGCTAACGTAACTATTTCCATCTCTTCGTTTAACTAAGTTGAATTTGCCCATACCTGATTCAATTGTAGTGTCTCCGCCTGTAGCATAGTCACCA',
       'TTGAATCAGGTATGGGCAAATTCAACTTAGTTAAACGAAGAGATGGAAATAGTTACGTTAGCATTCAAAGTTATGATTTATTGGCGGTAGGTTCTGATAA',
       'TTGAATAAATTGAATAAAATTACTAAATGTTTGTTTAACACTTTCAATTGCACCATTAACAAAATTTCTAAATGTTTCAGATTTCTTATAAGCAATTGTA',
       'TTATCCGAAATTT',
       'TTACAATTGCTTATAAGAAATCTGAAACATTTAGAAATTTTGTTAATGGTGCAATTGAAAGTGTTAAACAAACATTTAGTAATTTTATTCAATTTATTCA',
       'TTAATGGTGACTATGCTACAGGCGGAGACACTACAATTGAATCAGGTATGGGCAAATTCAACTTAGTTAAACGAAGAGATGGAAATAGTTACGTTAGCA

In [60]:
removed_features = X_train_features.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            6992, 6993, 6994, 6995, 6996, 6997, 6998, 6999, 7000, 7001],
           dtype='int64', length=6894)

In [61]:
len(removed_features)

6894

In [62]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 110) (255,) (255, 111)


In [63]:
# split into train, validation, and test data
from sklearn.model_selection import train_test_split
X_sel_train, X_sel_test, y_sel_train, y_sel_test = train_test_split(X_sel, y_sel,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel)

In [64]:
#### neural network
model_sel1 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [65]:
model_sel1.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [66]:
model_sel1.fit(X_sel_train, y_sel_train,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 605us/step - loss: 7.2068 - accuracy: 0.4719 - val_loss: 0.6950 - val_accuracy: 0.5844
Epoch 2/1000
178/178 [==============================] - 0s 110us/step - loss: 0.6955 - accuracy: 0.5000 - val_loss: 0.6943 - val_accuracy: 0.5844
Epoch 3/1000
178/178 [==============================] - 0s 181us/step - loss: 0.6982 - accuracy: 0.4944 - val_loss: 0.7045 - val_accuracy: 0.5714
Epoch 4/1000
178/178 [==============================] - 0s 182us/step - loss: 0.6978 - accuracy: 0.4831 - val_loss: 0.6875 - val_accuracy: 0.5325
Epoch 5/1000
178/178 [==============================] - 0s 163us/step - loss: 0.6979 - accuracy: 0.4719 - val_loss: 0.6942 - val_accuracy: 0.5325
Epoch 6/1000
178/178 [==============================] - 0s 175us/step - loss: 0.6719 - accuracy: 0.4944 - val_loss: 0.6842 - val_accuracy: 0.5584
Epoch 7/1000
178/178 [==============================] - 0s 139us/step - loss: 0

178/178 [==============================] - 0s 147us/step - loss: 0.5790 - accuracy: 0.7584 - val_loss: 0.6578 - val_accuracy: 0.5714
Epoch 57/1000
178/178 [==============================] - 0s 192us/step - loss: 0.5760 - accuracy: 0.7472 - val_loss: 0.6683 - val_accuracy: 0.5584
Epoch 58/1000
178/178 [==============================] - 0s 123us/step - loss: 0.5798 - accuracy: 0.7416 - val_loss: 0.6581 - val_accuracy: 0.5974
Epoch 59/1000
178/178 [==============================] - 0s 115us/step - loss: 0.5725 - accuracy: 0.7584 - val_loss: 0.6611 - val_accuracy: 0.5844
Epoch 60/1000
178/178 [==============================] - 0s 134us/step - loss: 0.5747 - accuracy: 0.7360 - val_loss: 0.6625 - val_accuracy: 0.5455
Epoch 61/1000
178/178 [==============================] - 0s 126us/step - loss: 0.5858 - accuracy: 0.7247 - val_loss: 0.6592 - val_accuracy: 0.5714
Epoch 62/1000
178/178 [==============================] - 0s 102us/step - loss: 0.5729 - accuracy: 0.7360 - val_loss: 0.6580 - val_ac

Epoch 112/1000
178/178 [==============================] - 0s 105us/step - loss: 0.5247 - accuracy: 0.7640 - val_loss: 0.6568 - val_accuracy: 0.5844
Epoch 113/1000
178/178 [==============================] - 0s 106us/step - loss: 0.5264 - accuracy: 0.7697 - val_loss: 0.6642 - val_accuracy: 0.5714
Epoch 114/1000
178/178 [==============================] - 0s 102us/step - loss: 0.5233 - accuracy: 0.7809 - val_loss: 0.6701 - val_accuracy: 0.5844
Epoch 115/1000
178/178 [==============================] - 0s 99us/step - loss: 0.5210 - accuracy: 0.7697 - val_loss: 0.6725 - val_accuracy: 0.5844
Epoch 116/1000
178/178 [==============================] - 0s 94us/step - loss: 0.5281 - accuracy: 0.7528 - val_loss: 0.6599 - val_accuracy: 0.5844
Epoch 117/1000
178/178 [==============================] - 0s 103us/step - loss: 0.5214 - accuracy: 0.7640 - val_loss: 0.6653 - val_accuracy: 0.5844
Epoch 118/1000
178/178 [==============================] - 0s 104us/step - loss: 0.5181 - accuracy: 0.7753 - val_lo

178/178 [==============================] - 0s 103us/step - loss: 0.4962 - accuracy: 0.7809 - val_loss: 0.6683 - val_accuracy: 0.5844
Epoch 168/1000
178/178 [==============================] - 0s 109us/step - loss: 0.4890 - accuracy: 0.7809 - val_loss: 0.6751 - val_accuracy: 0.5844
Epoch 169/1000
178/178 [==============================] - 0s 126us/step - loss: 0.4941 - accuracy: 0.7809 - val_loss: 0.6938 - val_accuracy: 0.5844
Epoch 170/1000
178/178 [==============================] - 0s 94us/step - loss: 0.4871 - accuracy: 0.7753 - val_loss: 0.6696 - val_accuracy: 0.5844
Epoch 171/1000
178/178 [==============================] - 0s 104us/step - loss: 0.5008 - accuracy: 0.7640 - val_loss: 0.6768 - val_accuracy: 0.5714
Epoch 172/1000
178/178 [==============================] - 0s 95us/step - loss: 0.4876 - accuracy: 0.7921 - val_loss: 0.6743 - val_accuracy: 0.5844
Epoch 173/1000
178/178 [==============================] - 0s 100us/step - loss: 0.4966 - accuracy: 0.7753 - val_loss: 0.7650 - va

178/178 [==============================] - 0s 101us/step - loss: 0.4773 - accuracy: 0.7978 - val_loss: 0.6795 - val_accuracy: 0.6104
Epoch 223/1000
178/178 [==============================] - 0s 112us/step - loss: 0.4733 - accuracy: 0.7921 - val_loss: 0.6865 - val_accuracy: 0.6104
Epoch 224/1000
178/178 [==============================] - 0s 102us/step - loss: 0.4843 - accuracy: 0.7809 - val_loss: 0.6930 - val_accuracy: 0.6104
Epoch 225/1000
178/178 [==============================] - 0s 96us/step - loss: 0.4646 - accuracy: 0.7865 - val_loss: 0.7539 - val_accuracy: 0.5844
Epoch 226/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4937 - accuracy: 0.7809 - val_loss: 0.6922 - val_accuracy: 0.6104
Epoch 227/1000
178/178 [==============================] - 0s 85us/step - loss: 0.4781 - accuracy: 0.7978 - val_loss: 0.6877 - val_accuracy: 0.6104
Epoch 228/1000
178/178 [==============================] - 0s 103us/step - loss: 0.4645 - accuracy: 0.7978 - val_loss: 0.7006 - val

...

178/178 [==============================] - 0s 110us/step - loss: 0.3718 - accuracy: 0.8371 - val_loss: 0.8032 - val_accuracy: 0.5974
Epoch 773/1000
178/178 [==============================] - 0s 97us/step - loss: 0.5537 - accuracy: 0.7303 - val_loss: 0.6909 - val_accuracy: 0.5844
Epoch 774/1000
178/178 [==============================] - 0s 107us/step - loss: 0.4524 - accuracy: 0.7978 - val_loss: 0.7138 - val_accuracy: 0.5974
Epoch 775/1000
178/178 [==============================] - 0s 96us/step - loss: 0.4360 - accuracy: 0.8034 - val_loss: 0.7235 - val_accuracy: 0.5844
Epoch 776/1000
178/178 [==============================] - 0s 93us/step - loss: 0.4139 - accuracy: 0.8202 - val_loss: 0.7439 - val_accuracy: 0.5974
Epoch 777/1000
178/178 [==============================] - 0s 94us/step - loss: 0.3771 - accuracy: 0.8258 - val_loss: 1.0615 - val_accuracy: 0.5974
Epoch 778/1000
178/178 [==============================] - 0s 100us/step - loss: 0.4190 - accuracy: 0.8258 - val_loss: 0.7104 - val_

178/178 [==============================] - 0s 106us/step - loss: 0.3844 - accuracy: 0.8483 - val_loss: 0.7417 - val_accuracy: 0.5974
Epoch 828/1000
178/178 [==============================] - 0s 109us/step - loss: 0.3630 - accuracy: 0.8371 - val_loss: 0.7633 - val_accuracy: 0.5974
Epoch 829/1000
178/178 [==============================] - 0s 91us/step - loss: 0.3604 - accuracy: 0.8371 - val_loss: 0.7644 - val_accuracy: 0.5974
Epoch 830/1000
178/178 [==============================] - 0s 103us/step - loss: 0.3870 - accuracy: 0.8258 - val_loss: 0.7982 - val_accuracy: 0.5844
Epoch 831/1000
178/178 [==============================] - 0s 120us/step - loss: 0.4224 - accuracy: 0.8146 - val_loss: 0.8231 - val_accuracy: 0.5714
Epoch 832/1000
178/178 [==============================] - 0s 109us/step - loss: 0.5252 - accuracy: 0.7809 - val_loss: 0.7584 - val_accuracy: 0.5844
Epoch 833/1000
178/178 [==============================] - 0s 102us/step - loss: 0.3897 - accuracy: 0.8202 - val_loss: 0.7084 - v

178/178 [==============================] - 0s 104us/step - loss: 0.4130 - accuracy: 0.8146 - val_loss: 0.6944 - val_accuracy: 0.6104
Epoch 883/1000
178/178 [==============================] - 0s 135us/step - loss: 0.4211 - accuracy: 0.8427 - val_loss: 0.7592 - val_accuracy: 0.6104
Epoch 884/1000
178/178 [==============================] - 0s 95us/step - loss: 0.4552 - accuracy: 0.8146 - val_loss: 0.7235 - val_accuracy: 0.6104
Epoch 885/1000
178/178 [==============================] - 0s 90us/step - loss: 0.3818 - accuracy: 0.8315 - val_loss: 0.7750 - val_accuracy: 0.6104
Epoch 886/1000
178/178 [==============================] - 0s 101us/step - loss: 0.5548 - accuracy: 0.7921 - val_loss: 0.7136 - val_accuracy: 0.6104
Epoch 887/1000
178/178 [==============================] - 0s 95us/step - loss: 0.4010 - accuracy: 0.8371 - val_loss: 0.6876 - val_accuracy: 0.6104
Epoch 888/1000
178/178 [==============================] - 0s 103us/step - loss: 0.3996 - accuracy: 0.8258 - val_loss: 0.7056 - val

178/178 [==============================] - 0s 103us/step - loss: 0.3866 - accuracy: 0.8090 - val_loss: 0.7227 - val_accuracy: 0.6104
Epoch 938/1000
178/178 [==============================] - 0s 100us/step - loss: 0.3914 - accuracy: 0.8146 - val_loss: 0.7298 - val_accuracy: 0.6104
Epoch 939/1000
178/178 [==============================] - 0s 99us/step - loss: 0.3660 - accuracy: 0.8427 - val_loss: 0.8445 - val_accuracy: 0.5974
Epoch 940/1000
178/178 [==============================] - 0s 120us/step - loss: 0.4089 - accuracy: 0.8202 - val_loss: 0.7333 - val_accuracy: 0.6104
Epoch 941/1000
178/178 [==============================] - 0s 94us/step - loss: 0.3836 - accuracy: 0.8427 - val_loss: 0.7648 - val_accuracy: 0.5974
Epoch 942/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3909 - accuracy: 0.8258 - val_loss: 0.7254 - val_accuracy: 0.5974
Epoch 943/1000
178/178 [==============================] - 0s 114us/step - loss: 0.4604 - accuracy: 0.8034 - val_loss: 0.7320 - val

178/178 [==============================] - 0s 108us/step - loss: 0.4202 - accuracy: 0.8315 - val_loss: 0.7052 - val_accuracy: 0.6104
Epoch 993/1000
178/178 [==============================] - 0s 108us/step - loss: 0.3758 - accuracy: 0.8427 - val_loss: 0.7252 - val_accuracy: 0.5974
Epoch 994/1000
178/178 [==============================] - 0s 105us/step - loss: 0.3776 - accuracy: 0.8483 - val_loss: 0.7000 - val_accuracy: 0.6104
Epoch 995/1000
178/178 [==============================] - 0s 108us/step - loss: 0.3865 - accuracy: 0.8315 - val_loss: 0.7113 - val_accuracy: 0.6104
Epoch 996/1000
178/178 [==============================] - 0s 96us/step - loss: 0.3738 - accuracy: 0.8315 - val_loss: 0.7231 - val_accuracy: 0.5974
Epoch 997/1000
178/178 [==============================] - 0s 92us/step - loss: 0.3766 - accuracy: 0.8315 - val_loss: 0.7218 - val_accuracy: 0.6104
Epoch 998/1000
178/178 [==============================] - 0s 107us/step - loss: 0.3739 - accuracy: 0.8371 - val_loss: 0.7277 - va

In [67]:
acc_test_sel1 = model_sel1.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel1*100))

77/77 [==============================] - 0s 134us/step
test accuracy: 61.04%


In [68]:
#### improve neural network
model_sel2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [69]:
model_sel2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [70]:
model_sel2.fit(X_sel_train, y_sel_train,
          batch_size=64, epochs=1000,
          validation_data=(X_sel_test, y_sel_test))

Train on 178 samples, validate on 77 samples
Epoch 1/1000
178/178 [==============================] - 0s 653us/step - loss: 5.0431 - accuracy: 0.5169 - val_loss: 16.7532 - val_accuracy: 0.5195
Epoch 2/1000
178/178 [==============================] - 0s 73us/step - loss: 11.8111 - accuracy: 0.5225 - val_loss: 1.5633 - val_accuracy: 0.5325
Epoch 3/1000
178/178 [==============================] - 0s 82us/step - loss: 1.5988 - accuracy: 0.5618 - val_loss: 0.6874 - val_accuracy: 0.5584
Epoch 4/1000
178/178 [==============================] - 0s 122us/step - loss: 0.6702 - accuracy: 0.6124 - val_loss: 0.6769 - val_accuracy: 0.5714
Epoch 5/1000
178/178 [==============================] - 0s 146us/step - loss: 0.6594 - accuracy: 0.6236 - val_loss: 0.6724 - val_accuracy: 0.5714
Epoch 6/1000
178/178 [==============================] - 0s 181us/step - loss: 0.6530 - accuracy: 0.6236 - val_loss: 0.6700 - val_accuracy: 0.5844
Epoch 7/1000
178/178 [==============================] - 0s 157us/step - loss: 0

178/178 [==============================] - 0s 135us/step - loss: 0.5936 - accuracy: 0.6910 - val_loss: 0.6601 - val_accuracy: 0.5714
Epoch 57/1000
178/178 [==============================] - 0s 136us/step - loss: 0.5928 - accuracy: 0.6798 - val_loss: 0.6579 - val_accuracy: 0.5714
Epoch 58/1000
178/178 [==============================] - 0s 174us/step - loss: 0.5922 - accuracy: 0.6966 - val_loss: 0.6583 - val_accuracy: 0.5584
Epoch 59/1000
178/178 [==============================] - 0s 118us/step - loss: 0.5908 - accuracy: 0.7191 - val_loss: 0.6590 - val_accuracy: 0.5714
Epoch 60/1000
178/178 [==============================] - 0s 106us/step - loss: 0.5885 - accuracy: 0.6966 - val_loss: 0.6580 - val_accuracy: 0.5844
Epoch 61/1000
178/178 [==============================] - 0s 137us/step - loss: 0.5879 - accuracy: 0.6910 - val_loss: 0.6596 - val_accuracy: 0.5584
Epoch 62/1000
178/178 [==============================] - 0s 102us/step - loss: 0.5867 - accuracy: 0.7079 - val_loss: 0.6597 - val_ac

178/178 [==============================] - 0s 79us/step - loss: 0.5591 - accuracy: 0.7528 - val_loss: 0.6705 - val_accuracy: 0.5584
Epoch 113/1000
178/178 [==============================] - 0s 93us/step - loss: 0.5589 - accuracy: 0.7416 - val_loss: 0.6708 - val_accuracy: 0.5584
Epoch 114/1000
178/178 [==============================] - 0s 60us/step - loss: 0.5566 - accuracy: 0.7472 - val_loss: 0.6684 - val_accuracy: 0.5584
Epoch 115/1000
178/178 [==============================] - 0s 89us/step - loss: 0.5574 - accuracy: 0.7584 - val_loss: 0.6720 - val_accuracy: 0.5455
Epoch 116/1000
178/178 [==============================] - 0s 73us/step - loss: 0.5566 - accuracy: 0.7584 - val_loss: 0.6693 - val_accuracy: 0.5455
Epoch 117/1000
178/178 [==============================] - 0s 58us/step - loss: 0.5688 - accuracy: 0.7584 - val_loss: 0.6824 - val_accuracy: 0.5714
Epoch 118/1000
178/178 [==============================] - 0s 92us/step - loss: 0.5633 - accuracy: 0.7360 - val_loss: 0.6699 - val_acc

Epoch 168/1000
178/178 [==============================] - 0s 75us/step - loss: 0.5289 - accuracy: 0.7809 - val_loss: 0.6721 - val_accuracy: 0.5974
Epoch 169/1000
178/178 [==============================] - 0s 71us/step - loss: 0.5313 - accuracy: 0.7697 - val_loss: 0.6736 - val_accuracy: 0.5844
Epoch 170/1000
178/178 [==============================] - 0s 102us/step - loss: 0.5288 - accuracy: 0.7809 - val_loss: 0.6745 - val_accuracy: 0.5844
Epoch 171/1000
178/178 [==============================] - 0s 85us/step - loss: 0.5293 - accuracy: 0.7753 - val_loss: 0.6730 - val_accuracy: 0.5844
Epoch 172/1000
178/178 [==============================] - 0s 90us/step - loss: 0.5338 - accuracy: 0.7753 - val_loss: 0.6740 - val_accuracy: 0.5844
Epoch 173/1000
178/178 [==============================] - 0s 67us/step - loss: 0.5279 - accuracy: 0.7753 - val_loss: 0.6721 - val_accuracy: 0.5844
Epoch 174/1000
178/178 [==============================] - 0s 79us/step - loss: 0.5358 - accuracy: 0.7528 - val_loss: 

178/178 [==============================] - 0s 175us/step - loss: 0.5162 - accuracy: 0.7809 - val_loss: 0.6654 - val_accuracy: 0.5844
Epoch 224/1000
178/178 [==============================] - 0s 119us/step - loss: 0.5157 - accuracy: 0.7865 - val_loss: 0.6712 - val_accuracy: 0.5844
Epoch 225/1000
178/178 [==============================] - 0s 121us/step - loss: 0.5138 - accuracy: 0.7753 - val_loss: 0.6769 - val_accuracy: 0.5844
Epoch 226/1000
178/178 [==============================] - 0s 110us/step - loss: 0.5267 - accuracy: 0.7640 - val_loss: 0.6677 - val_accuracy: 0.5844
Epoch 227/1000
178/178 [==============================] - 0s 103us/step - loss: 0.5144 - accuracy: 0.7809 - val_loss: 0.6670 - val_accuracy: 0.5844
Epoch 228/1000
178/178 [==============================] - 0s 112us/step - loss: 0.5192 - accuracy: 0.7753 - val_loss: 0.6763 - val_accuracy: 0.5844
Epoch 229/1000
178/178 [==============================] - 0s 133us/step - loss: 0.5399 - accuracy: 0.7528 - val_loss: 0.6644 - 

...

Epoch 725/1000
178/178 [==============================] - 0s 62us/step - loss: 0.4415 - accuracy: 0.7865 - val_loss: 0.8126 - val_accuracy: 0.5844
Epoch 726/1000
178/178 [==============================] - 0s 102us/step - loss: 0.4801 - accuracy: 0.7865 - val_loss: 0.7367 - val_accuracy: 0.5974
Epoch 727/1000
178/178 [==============================] - 0s 98us/step - loss: 0.4417 - accuracy: 0.8034 - val_loss: 0.8446 - val_accuracy: 0.5974
Epoch 728/1000
178/178 [==============================] - 0s 82us/step - loss: 0.4670 - accuracy: 0.8090 - val_loss: 0.7067 - val_accuracy: 0.6104
Epoch 729/1000
178/178 [==============================] - 0s 69us/step - loss: 0.4124 - accuracy: 0.8034 - val_loss: 0.7118 - val_accuracy: 0.6104
Epoch 730/1000
178/178 [==============================] - 0s 80us/step - loss: 0.4061 - accuracy: 0.8146 - val_loss: 0.7210 - val_accuracy: 0.6234
Epoch 731/1000
178/178 [==============================] - 0s 84us/step - loss: 0.4178 - accuracy: 0.8258 - val_loss: 

178/178 [==============================] - 0s 84us/step - loss: 0.4084 - accuracy: 0.8146 - val_loss: 0.7068 - val_accuracy: 0.6104
Epoch 781/1000
178/178 [==============================] - 0s 75us/step - loss: 0.3977 - accuracy: 0.8315 - val_loss: 0.7099 - val_accuracy: 0.6104
Epoch 782/1000
178/178 [==============================] - 0s 97us/step - loss: 0.4018 - accuracy: 0.8146 - val_loss: 0.8060 - val_accuracy: 0.5974
Epoch 783/1000
178/178 [==============================] - 0s 65us/step - loss: 0.4589 - accuracy: 0.7865 - val_loss: 0.7133 - val_accuracy: 0.6104
Epoch 784/1000
178/178 [==============================] - 0s 84us/step - loss: 0.4203 - accuracy: 0.8146 - val_loss: 0.7137 - val_accuracy: 0.6104
Epoch 785/1000
178/178 [==============================] - 0s 80us/step - loss: 0.4252 - accuracy: 0.8090 - val_loss: 0.8336 - val_accuracy: 0.5974
Epoch 786/1000
178/178 [==============================] - 0s 76us/step - loss: 0.4299 - accuracy: 0.8202 - val_loss: 0.7088 - val_acc

Epoch 836/1000
178/178 [==============================] - 0s 61us/step - loss: 0.4543 - accuracy: 0.8146 - val_loss: 0.7103 - val_accuracy: 0.6104
Epoch 837/1000
178/178 [==============================] - 0s 77us/step - loss: 0.4098 - accuracy: 0.8090 - val_loss: 0.7138 - val_accuracy: 0.6104
Epoch 838/1000
178/178 [==============================] - 0s 76us/step - loss: 0.4454 - accuracy: 0.7809 - val_loss: 0.7254 - val_accuracy: 0.6104
Epoch 839/1000
178/178 [==============================] - ETA: 0s - loss: 0.3716 - accuracy: 0.87 - 0s 69us/step - loss: 0.4243 - accuracy: 0.8146 - val_loss: 0.7219 - val_accuracy: 0.6104
Epoch 840/1000
178/178 [==============================] - 0s 91us/step - loss: 0.4000 - accuracy: 0.8202 - val_loss: 0.7128 - val_accuracy: 0.6104
Epoch 841/1000
178/178 [==============================] - 0s 85us/step - loss: 0.3989 - accuracy: 0.8146 - val_loss: 0.7140 - val_accuracy: 0.6104
Epoch 842/1000
178/178 [==============================] - 0s 73us/step - los

178/178 [==============================] - 0s 83us/step - loss: 0.3982 - accuracy: 0.8202 - val_loss: 0.7358 - val_accuracy: 0.5974
Epoch 892/1000
178/178 [==============================] - 0s 67us/step - loss: 0.3876 - accuracy: 0.8371 - val_loss: 0.7177 - val_accuracy: 0.6104
Epoch 893/1000
178/178 [==============================] - 0s 80us/step - loss: 0.4029 - accuracy: 0.8034 - val_loss: 0.7213 - val_accuracy: 0.6104
Epoch 894/1000
178/178 [==============================] - 0s 104us/step - loss: 0.4013 - accuracy: 0.8258 - val_loss: 0.7712 - val_accuracy: 0.6104
Epoch 895/1000
178/178 [==============================] - 0s 73us/step - loss: 0.4133 - accuracy: 0.7921 - val_loss: 0.7390 - val_accuracy: 0.5974
Epoch 896/1000
178/178 [==============================] - 0s 78us/step - loss: 0.3835 - accuracy: 0.8315 - val_loss: 0.7126 - val_accuracy: 0.6104
Epoch 897/1000
178/178 [==============================] - 0s 84us/step - loss: 0.3814 - accuracy: 0.8202 - val_loss: 0.7714 - val_ac

Epoch 947/1000
178/178 [==============================] - 0s 90us/step - loss: 0.3951 - accuracy: 0.8146 - val_loss: 0.7198 - val_accuracy: 0.6104
Epoch 948/1000
178/178 [==============================] - 0s 186us/step - loss: 0.3722 - accuracy: 0.8483 - val_loss: 0.7200 - val_accuracy: 0.6104
Epoch 949/1000
178/178 [==============================] - 0s 157us/step - loss: 0.3705 - accuracy: 0.8371 - val_loss: 0.7844 - val_accuracy: 0.6104
Epoch 950/1000
178/178 [==============================] - 0s 169us/step - loss: 0.4469 - accuracy: 0.7921 - val_loss: 0.7219 - val_accuracy: 0.6104
Epoch 951/1000
178/178 [==============================] - 0s 114us/step - loss: 0.3854 - accuracy: 0.8315 - val_loss: 0.7180 - val_accuracy: 0.6104
Epoch 952/1000
178/178 [==============================] - 0s 77us/step - loss: 0.3786 - accuracy: 0.8315 - val_loss: 0.7204 - val_accuracy: 0.6104
Epoch 953/1000
178/178 [==============================] - 0s 87us/step - loss: 0.3715 - accuracy: 0.8483 - val_los

In [71]:
acc_test_sel2 = model_sel2.evaluate(X_sel_test, y_sel_test)[1]
print('test accuracy: %.2f%%' % (acc_test_sel2*100))

77/77 [==============================] - 0s 232us/step
test accuracy: 61.04%


In [72]:
############ Logistic Regression ############

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [74]:
###### logistics
log = LogisticRegression()
log.fit(X_train, y_train)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [75]:
y_pred = log.predict(X_test)
print('combination logistic test accuracy %.2f%%' % (log.score(X_test, y_test)*100))

combination logistic test accuracy 61.04%


In [76]:
############## Random Forest ##############

In [77]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [78]:
from sklearn.metrics import accuracy_score
y_pred = rf.predict(X_test)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test, y_pred)*100))

combination test accuracy: 63.64%


In [79]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [80]:
## random forest model with CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv = RandomForestClassifier(n_estimators=100, random_state=123)

accs = cross_val_score(estimator=rfcv, X=X_train, y=y_train, cv=5)
print(accs)
print(accs.mean())

[0.83333333 0.80555556 0.69444444 0.80555556 0.73529412]
0.7748366013071895
